In [7]:
from data import loader, exporter
from constant import *
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [15]:
df_basic = loader.to_concat_df('XW_ENTINFO_BASIC')
df_basic

,DATA_DAT,CUST_NO,OPTO,OPFROM,ENTSTATUS,REGCAP,ESDATE,FRNAME,ENTTYPE_CD,REGPROVIN_CD,INDS_CD,SRC
0,20020727,182d6a854532dd26a1b111e77bd501f4,20420701,19920702,在营（开业）,275.77,19920702,444360f253c09f7d97a3b15bb26a8573,46f6ddc7a540fa9e2b5ac3fa24038304,3048e339d9689928fc83eb6aa552ccfb,1407ccc0c9f66ff0402271dada75885e,train
1,20020727,f60def7aa5dc124ddae552b7bf5c7675,长期,19930512,在营（开业）,218.88,19930512,0dbf92ddf037b7727060924e76284d1c,46f6ddc7a540fa9e2b5ac3fa24038304,181b1987746f41b780200a407686ffc5,d5f34ccd26f45e66747884462c45c309,train
2,20020727,87bcf5ea6b87ab5da08931dd45dfdc1e,20200831,20000901,在营（开业）,1280.00,20000901,45dc91c39f9bceb2bb0675d263755273,6475fff732caa245e3895f0884ba2912,85ff7de4fd2d0198216d0f0aaa20a1a8,5a4d3096e0c6ba990652ae43f4bfaafe,train
3,20020727,96d8793d27b9361053f5d7f2b81f4831,长期,19980918,在营（开业）,240.10,19980918,f2ebb9f19232a67d47b91fba707aeb73,d6c937931560c340740515da55cfabb5,ecb1080b93551a426f2b701a1accf2e6,e60392183298b9ecb8a034d6cfc08a84,train
4,20020727,8c2f07a8bbff12ee7bb276cf473fe3d2,长期,19950512,在营（开业）,128.00,19950512,2e736490f260d55acecb58bc1dd4d69c,46f6ddc7a540fa9e2b5ac3fa24038304,56ed722fc914c13fcf88155f44962637,e1755eeb8a16ea198bd60b3923ab73e8,train
...,...,...,...,...,...,...,...,...,...,...,...,...
5980,20020827,6b94f8df43a0135ebff03f41aa972592,长期,19981117,在营（开业）,128.00,19981117,4d03460029dc516dad10e28633a90ecc,d6c937931560c340740515da55cfabb5,ba6719ada4ed813585ac0a20e6da0505,421da701c180a70a4ca99431f6271287,test
5981,20020827,c5ff1b26c516179279cf7f3dff8909be,20090424,19890425,在营（开业）,128.00,19890425,08f72759fafa32337b9f3a483bdb6b5c,46f6ddc7a540fa9e2b5ac3fa24038304,7980f07be94b456d28e5c88ce2896af0,421b4ed6a7b833c1cff6316827d4d87a,test
5982,20020827,55da75cdfe3ec1e7999eb747a477883b,长期,19980609,在营（开业）,397.73,19980609,730f3eb3da602a1635f416b710ccb6fc,d6c937931560c340740515da55cfabb5,85ff7de4fd2d0198216d0f0aaa20a1a8,a6335a19efadadfe1e1cf7a9e44555d7,test
5983,20020827,e9d38c6dae16216e8e59d0f58ea5c9ea,长期,19951001,在营（开业）,101.59,19951001,79a09f3360402c715e7aa6d94ed1d991,d6c937931560c340740515da55cfabb5,890dd4a3e832ce5d449f82d89a53b37d,3855496272a4b9bb0a46d8d74184233c,test


In [23]:
def construct_features_basic(df):
    # 1. 计算企业存续时间
    df['OPFROM'] = pd.to_datetime(df['OPFROM'], format='%Y%m%d', errors='coerce')
    df['OPTO'] = df['OPTO'].replace('长期', '2050-12-31')  # 将“长期”替换为一个大日期
    df['OPTO'] = pd.to_datetime(df['OPTO'], format='%Y%m%d', errors='coerce')
    df['LIFE_SPAN'] = (df['OPTO'] - df['OPFROM']).dt.days / 365.0  # 存续时间（年）
    df['DATA_DAT'] = pd.to_datetime(df['DATA_DAT'], format='%Y%m%d', errors='coerce')
    df['DAYS_TO_EXPIRE'] = (df['OPTO'] - df['DATA_DAT']).dt.days / 365.0  # 截至DATA_DAT，还有多久到期

    # 2. 企业经营状态独热编码
    df = pd.get_dummies(df, columns=['ENTSTATUS'], prefix='ENTSTATUS', drop_first=True)

    # 3. 企业类型编码
    label_encoder_enttype = LabelEncoder()
    df['ENTTYPE_CD_ENCODED'] = label_encoder_enttype.fit_transform(df['ENTTYPE_CD'].astype(str))

    # 4. 企业地区编码
    label_encoder_regprov = LabelEncoder()
    df['REGPROVIN_CD_ENCODED'] = label_encoder_regprov.fit_transform(df['REGPROVIN_CD'].astype(str))

    # 5. 行业类别编码
    label_encoder_inds = LabelEncoder()
    df['INDS_CD_ENCODED'] = label_encoder_inds.fit_transform(df['INDS_CD'].astype(str))

    df = df.drop(columns=['OPTO', 'OPFROM', 'DATA_DAT', 'FRNAME', 'ENTTYPE_CD', 'REGPROVIN_CD', 'INDS_CD', 'ESDATE','SRC'])

    return df


df_basic = loader.to_concat_df('XW_ENTINFO_BASIC')
df_basic_features = construct_features_basic(df_basic)

exporter.export_df_to_preprocess('basic', df_basic_features)

df_basic_features

,CUST_NO,REGCAP,LIFE_SPAN,DAYS_TO_EXPIRE,ENTSTATUS_在营（开业）,ENTSTATUS_注销,ENTSTATUS_迁出,ENTTYPE_CD_ENCODED,REGPROVIN_CD_ENCODED,INDS_CD_ENCODED
0,182d6a854532dd26a1b111e77bd501f4,275.77,50.030137,39.956164,1,0,0,20,4,117
1,f60def7aa5dc124ddae552b7bf5c7675,218.88,NaN,NaN,1,0,0,20,2,1223
2,87bcf5ea6b87ab5da08931dd45dfdc1e,1280.00,20.010959,18.109589,1,0,0,31,13,506
3,96d8793d27b9361053f5d7f2b81f4831,240.10,NaN,NaN,1,0,0,56,28,1308
4,8c2f07a8bbff12ee7bb276cf473fe3d2,128.00,NaN,NaN,1,0,0,20,6,1285
...,...,...,...,...,...,...,...,...,...,...
5980,6b94f8df43a0135ebff03f41aa972592,128.00,NaN,NaN,1,0,0,56,21,359
5981,c5ff1b26c516179279cf7f3dff8909be,128.00,20.010959,6.663014,1,0,0,20,12,358
5982,55da75cdfe3ec1e7999eb747a477883b,397.73,NaN,NaN,1,0,0,56,13,931
5983,e9d38c6dae16216e8e59d0f58ea5c9ea,101.59,NaN,NaN,1,0,0,56,14,303


In [22]:
# 统计空值情况
missing_info = df_basic_features.isnull().sum() / len(df_basic_features) * 100

# 统计各个特征的基本分布
distribution_info = df_basic_features.describe(include='all')

# 返回缺失值信息和分布信息
missing_info, distribution_info

(CUST_NO                  0.000000
 REGCAP                   0.081247
 SRC                      0.000000
 LIFE_SPAN               68.230674
 DAYS_TO_EXPIRE          68.230674
 ENTSTATUS_在营（开业）         0.000000
 ENTSTATUS_注销             0.000000
 ENTSTATUS_迁出             0.000000
 ENTTYPE_CD_ENCODED       0.000000
 REGPROVIN_CD_ENCODED     0.000000
 INDS_CD_ENCODED          0.000000
 dtype: float64,
                                  CUST_NO        REGCAP    SRC     LIFE_SPAN  \
 count                              59079  59031.000000  59079  18769.000000   
 unique                             59079           NaN      2           NaN   
 top     182d6a854532dd26a1b111e77bd501f4           NaN  train           NaN   
 freq                                   1           NaN  53094           NaN   
 mean                                 NaN    229.934970    NaN     27.608998   
 std                                  NaN    166.184103    NaN     12.490971   
 min                                  